In [85]:
def get_pixels(file_name: str) -> list[tuple[int]]:
    with open(file_name, "r") as f:
        line = f.readline().strip()
        elms = line.split(" ")
        pixels = []
        r, g = None, None
        for elm in elms:
            if r is None:
                r = elm
            elif g is None:
                g = elm
            else:
                pixels.append((int(r), int(g), int(elm)))
                r, g = None, None
        return pixels

In [86]:
# テキストファイルを改行した結果と照らし合わせる
pixels = get_pixels("./data/image1.txt")
with open("out/pixels", mode="w") as f:
    f.write(str(pixels))
assert len(pixels) == 388800 / 3

## (2)


In [15]:
# とりあえず実験として、直前の(255, 255, 255)から次の(255, 255, 255)の間隔がどうなっているかを見る。
pixels = get_pixels("./data/image1.txt")

rows = []
width = 0
for i, pixel in enumerate(pixels):
    if pixel[0] == 255 and pixel[1] == 255 and pixel[2] == 255:
        rows.append({"width": width, "to": i})
        width = 0
    else:
        width += 1

print(rows)

[{'width': 479, 'to': 479}, {'width': 479, 'to': 959}, {'width': 479, 'to': 1439}, {'width': 479, 'to': 1919}, {'width': 479, 'to': 2399}, {'width': 479, 'to': 2879}, {'width': 479, 'to': 3359}, {'width': 479, 'to': 3839}, {'width': 479, 'to': 4319}, {'width': 479, 'to': 4799}, {'width': 479, 'to': 5279}, {'width': 479, 'to': 5759}, {'width': 479, 'to': 6239}, {'width': 479, 'to': 6719}, {'width': 479, 'to': 7199}, {'width': 479, 'to': 7679}, {'width': 479, 'to': 8159}, {'width': 479, 'to': 8639}, {'width': 479, 'to': 9119}, {'width': 479, 'to': 9599}, {'width': 479, 'to': 10079}, {'width': 479, 'to': 10559}, {'width': 479, 'to': 11039}, {'width': 479, 'to': 11519}, {'width': 479, 'to': 11999}, {'width': 479, 'to': 12479}, {'width': 479, 'to': 12959}, {'width': 479, 'to': 13439}, {'width': 479, 'to': 13919}, {'width': 479, 'to': 14399}, {'width': 479, 'to': 14879}, {'width': 479, 'to': 15359}, {'width': 479, 'to': 15839}, {'width': 479, 'to': 16319}, {'width': 479, 'to': 16799}, {'widt

In [19]:
# おそらく479ピクセルだが、念のためそれ以上の値がないか確かめる
max = 479
for row in rows:
    if row["width"] > max:
        max = row["width"]
print(max)

479


## (3)


In [21]:
# とりあえず明るさを配列に加える
pixels = get_pixels("./data/image1.txt")

brightnesses = []
for i, pixel in enumerate(pixels):
    brightnesses.append((i, pixel[0] ** 2 + pixel[1] ** 2 + pixel[2] ** 2))

# ファイルに書き出す。これは検証でも使える。
with open("out/brightnesses", mode="w") as f:
    f.write("\n".join([str(brightness) for brightness in brightnesses]))

In [22]:
255**2 * 3 == 195075

True

In [26]:
# ソートして中央の値を取る
sorted_brightnesses = sorted(brightnesses, key=lambda x: x[1])
print(sorted_brightnesses[len(sorted_brightnesses) // 2])

# 目視でも確認する。
with open("out/sorted_brightnesses", mode="w") as f:
    f.write("\n".join([str(brightness) for brightness in sorted_brightnesses]))

(11997, 79896)


## (4)


In [18]:
def append_index_and_brightness(pixels: list[tuple[int]]) -> list[tuple[int]]:
    with_index_and_brightness = []
    for i, pixel in enumerate(pixels):
        with_index_and_brightness.append(
            (
                i,
                pixel[0],
                pixel[1],
                pixel[2],
                pixel[0] ** 2 + pixel[1] ** 2 + pixel[2] ** 2,
            )
        )

    return sorted(with_index_and_brightness, key=lambda x: x[4])

In [19]:
origin = [(1, 1, 1), (0, 0, 0)]
expected = [(1, 0, 0, 0, 0), (0, 1, 1, 1, 3)]
actual = append_index_and_brightness(origin)
print(actual)
assert expected == actual

[(1, 0, 0, 0, 0), (0, 1, 1, 1, 3)]


In [20]:
# とりあえず整形したデータを眺める
pixels2 = get_pixels("./data/image2.txt")
with open("out/pixels2", mode="w") as f:
    f.write("\n".join([str(pixel2) for pixel2 in pixels2]))

sorted_brightnesses2 = append_index_and_brightness(pixels2)
print(len(sorted_brightnesses2))

NameError: name 'get_pixels' is not defined

In [71]:
def get_initial_cluster_core(pixels, k):
    cores = []
    n = len(pixels)
    for i in range(k):
        cores.append({"core": pixels[n * i // k], "members": []})
    return cores

In [72]:
k = 4
actual = get_initial_cluster_core(sorted_brightnesses2, k)
print(actual)

NameError: name 'sorted_brightnesses2' is not defined

## (5) k 近傍法


In [73]:
# 複雑なのでテストがあった方が良い。とりあえず、2パターン考えよう。
# ケース1: 黒ピクセル500, 白ピクセル500の画像。k=100とすると、i=25ではまず黒, i=75ではまず白になるはずだ
# → これは誤りだった（何も分類されないクラスターができてバグの原因になる）
# ケース2: ランダムな0~255の値を取る白黒画像。十分なピクセル数があればi=k/2で(128, 128, 128)くらいになるはずだ。

In [74]:
def testcase1():
    pixels = []
    for _ in range(0, 500):
        pixels.append((0, 0, 0))
        pixels.append((255, 255, 255))
    return pixels


testcase1_pixels = testcase1()
with open("out/testcase1_pixels", mode="w") as f:
    f.write("\n".join([str(pixel) for pixel in testcase1_pixels]))

In [75]:
import random


def testcase2():
    pixels = []
    for _ in range(0, 100000):
        brightness = random.randint(0, 255)
        pixels.append((brightness, brightness, brightness))
    return pixels


testcase2_pixels = testcase2()
with open("out/testcase2_pixels", mode="w") as f:
    f.write("\n".join([str(pixel) for pixel in testcase2_pixels]))

In [76]:
def get_distance(pix1, pix2) -> int:
    _, r1, g1, b1, _ = pix1
    _, r2, g2, b2, _ = pix2
    return abs(r1 - r2) + abs(g1 - g2) + abs(b1 - b2)

In [77]:
expected = 30
actual = get_distance(
    (0, 100, 100, 100, 300000), (1, 100, 90, 120, 100**2 + 90**2 + 120**2)
)
assert actual == expected

In [78]:
def member_updated(clusters, members):
    MAX_INT = 2**63 - 1

    assigned = [{"core": cluster["core"], "members": []} for cluster in clusters]
    for pixel in members:
        champion = -1
        champion_distance = MAX_INT
        for index, cluster in enumerate(assigned):
            distance = get_distance(pixel, cluster["core"])
            if distance <= champion_distance:
                champion = index
                champion_distance = distance

        assigned[champion]["members"].append(pixel)
    return assigned

In [79]:
original_members = [
    (0, 0, 0, 0, 0),
    (0, 1, 1, 1, 3),
    (0, 100, 100, 100, 30000),
    (0, 100, 100, 100, 30000),
]
original_clusters = [
    {
        "core": (0, 1, 1, 1, 3),
        "members": [(0, 100, 100, 100, 30000), (0, 100, 100, 100, 30000)],
    },
    {"core": (0, 100, 100, 100, 30000), "members": [(0, 0, 0, 0, 0), (0, 1, 1, 1, 3)]},
]
expected = clusters = [
    {"core": (0, 1, 1, 1, 3), "members": [(0, 0, 0, 0, 0), (0, 1, 1, 1, 3)]},
    {
        "core": (0, 100, 100, 100, 30000),
        "members": [(0, 100, 100, 100, 30000), (0, 100, 100, 100, 30000)],
    },
]
actual = member_updated(original_clusters, original_members)
print(actual)
assert actual == expected

[{'core': (0, 1, 1, 1, 3), 'members': [(0, 0, 0, 0, 0), (0, 1, 1, 1, 3)]}, {'core': (0, 100, 100, 100, 30000), 'members': [(0, 100, 100, 100, 30000), (0, 100, 100, 100, 30000)]}]


In [80]:
def core_updated(cluster):
    MAX_INT = 2**63 - 1
    members = cluster["members"]
    r_total, g_total, b_total = -1, -1, -1
    for member in members:
        _, r, g, b, _ = member
        r_total += r
        g_total += g
        b_total += b
    r_c, g_c, b_c = (
        r_total / len(members),
        g_total / len(members),
        b_total / len(members),
    )

    champion = None
    champion_dist = MAX_INT
    for member in members:
        dist = get_distance((-1, r_c, g_c, b_c, -1), member)
        # print(f"{champion=}, {champion_dist=}, {member=}, {dist=}")
        if dist <= champion_dist:
            champion = member
            champion_dist = dist

    return {"core": champion, "members": members}

In [81]:
origin = {
    "core": (0, 0, 0, 0, 0),
    "members": [(0, 0, 0, 0, 0), (0, 100, 100, 100, 30000), (0, 200, 200, 200, 120000)],
}
expected = {
    "core": (0, 100, 100, 100, 30000),
    "members": [(0, 0, 0, 0, 0), (0, 100, 100, 100, 30000), (0, 200, 200, 200, 120000)],
}
actual = core_updated(origin)

assert actual == expected

In [82]:
# TODO: tqdmインストールしておく


def k_neighbor(pixels: list[tuple[int]], k: int, iteration: int):
    with_index_and_brightness = append_index_and_brightness(pixels)
    clusters = get_initial_cluster_core(with_index_and_brightness, k)
    # print(clusters)

    for _ in range(iteration):
        member_updated_clusters = member_updated(clusters, with_index_and_brightness)
        clusters = [core_updated(cluster) for cluster in member_updated_clusters]

    return clusters

In [83]:
k_neighbor_clusters = k_neighbor(testcase2_pixels, 128, 3)
with open("out/k_neighbor", mode="w") as f:
    f.write("\n".join([str(cluster) for cluster in k_neighbor_clusters]))

In [88]:
pixels2 = get_pixels("./data/image2.txt")
sorted_brightnesses2 = append_index_and_brightness(pixels2)
k_neighbor_clusters2 = k_neighbor(sorted_brightnesses2, 128, 10)
with open("out/k_neighbor_clusters2", mode="w") as f:
    f.write("\n".join([str(cluster) for cluster in k_neighbor_clusters2]))

KeyboardInterrupt: 